In [3]:
import csv
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC

from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import GaussianNB  # Naive bayes
from sklearn.neighbors import KNeighborsClassifier

#Exploracion de datos

Se cargan los datos

In [4]:
yes_or_no = pd.read_excel("DatosHito2.xlsx")
yes_or_no.head()

,comment,subreddit,score,sarcasm,Llave,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,"I don't pay attention to her, but as long as s...",AskReddit,0,yes,AskReddityes,,AskReddityes,65674.0,0.724415,NaN,sarcasm,170516.0,0.577594
1,Trick or treating in general is just weird...,AskReddit,1,yes,AskReddityes,NaN,AskRedditno,24984.0,0.275585,NaN,nosarcasm,124702.0,0.422406
2,what the fuck,AskReddit,22,yes,AskReddityes,NaN,NaN,90658.0,NaN,NaN,NaN,295218.0,NaN
3,This would make me cry.,AskReddit,1,yes,AskReddityes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,My stuffed animal I've had since I was born.,AskReddit,1,yes,AskReddityes,NaN,funnyyes,17939.0,0.418149,NaN,NaN,NaN,NaN


In [5]:
yes_or_no.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295218 entries, 0 to 295217
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   comment      295218 non-null  object 
 1   subreddit    295218 non-null  object 
 2   score        295218 non-null  int64  
 3   sarcasm      295218 non-null  object 
 4   Llave        295218 non-null  object 
 5   Unnamed: 5   1 non-null       object 
 6   Unnamed: 6   10 non-null      object 
 7   Unnamed: 7   15 non-null      float64
 8   Unnamed: 8   10 non-null      float64
 9   Unnamed: 9   0 non-null       float64
 10  Unnamed: 10  2 non-null       object 
 11  Unnamed: 11  3 non-null       float64
 12  Unnamed: 12  2 non-null       float64
dtypes: float64(5), int64(1), object(7)
memory usage: 29.3+ MB


In [6]:
yes_or_no["subreddit"].value_counts()

AskReddit          90658
politics           64392
worldnews          51288
leagueoflegends    45979
funny              42901
Name: subreddit, dtype: int64

In [7]:
cantAskRedditYes=len(yes_or_no[(yes_or_no['subreddit']=="AskReddit")& (yes_or_no['sarcasm']=="yes")])
cantAskRedditNo=len(yes_or_no[(yes_or_no['subreddit']=="AskReddit")& (yes_or_no['sarcasm']=="no")])

cantFunnyYes=len(yes_or_no[(yes_or_no['subreddit']=="funny")& (yes_or_no['sarcasm']=="yes")])
cantFunnyNo=len(yes_or_no[(yes_or_no['subreddit']=="funny")& (yes_or_no['sarcasm']=="no")])

cantPoliticsYes=len(yes_or_no[(yes_or_no['subreddit']=="politics")& (yes_or_no['sarcasm']=="yes")])
cantPoliticsNo=len(yes_or_no[(yes_or_no['subreddit']=="politics")& (yes_or_no['sarcasm']=="no")])

cantLeagueOfLegendsYes=len(yes_or_no[(yes_or_no['subreddit']=="leagueoflegends")& (yes_or_no['sarcasm']=="yes")])
cantLeagueOfLegendsNo=len(yes_or_no[(yes_or_no['subreddit']=="leagueoflegends")& (yes_or_no['sarcasm']=="no")])

cantWorldNewsYes=len(yes_or_no[(yes_or_no['subreddit']=="worldnews")& (yes_or_no['sarcasm']=="yes")])
cantWorldNewsNo=len(yes_or_no[(yes_or_no['subreddit']=="worldnews")& (yes_or_no['sarcasm']=="no")])

cantAskReddit=cantAskRedditYes+cantAskRedditNo
cantFunny=cantFunnyYes+cantFunnyNo
cantPolitics=cantPoliticsYes+cantPoliticsNo
cantLeagueOfLegends=cantLeagueOfLegendsYes+cantLeagueOfLegendsNo
cantWorldNews=cantWorldNewsYes+cantWorldNewsNo

print("Proporción comentarios sarcasticos en AskReddit: ", cantAskRedditYes/cantAskReddit)
print("Proporción comentarios no sarcasticos en AskReddit: ", cantAskRedditNo/cantAskReddit)

print("Proporción comentarios sarcasticos en funny: ", cantFunnyYes/cantFunny)
print("Proporción comentarios no sarcasticos en funn: ", cantFunnyNo/cantFunny)

print("Proporción comentarios sarcasticos en politics: ", cantPoliticsYes/cantPolitics)
print("Proporción comentarios no sarcasticos en politics: ", cantPoliticsNo/cantPolitics)

print("Proporción comentarios sarcasticos en leagueoflegends: ", cantLeagueOfLegendsYes/cantLeagueOfLegends)
print("Proporción comentarios no sarcasticos en leagueoflegends: ", cantLeagueOfLegendsNo/cantLeagueOfLegends)

print("Proporción comentarios sarcasticos en worldnews: ", cantWorldNewsYes/cantWorldNews)
print("Proporción comentarios no sarcasticos en worldnews: ", cantWorldNewsNo/cantWorldNews)


cantSarcastica=cantAskRedditYes+cantFunnyYes+cantPoliticsYes+cantLeagueOfLegendsYes+cantWorldNewsYes
cantNotSarcastica=cantAskRedditNo+cantFunnyNo+cantPoliticsNo+cantLeagueOfLegendsNo+cantWorldNewsNo
cantTotal=cantSarcastica+cantNotSarcastica

print("Proporción comentarios sarcasticos en el dataset: ", cantSarcastica/cantTotal)
print("Proporción comentarios no sarcasticos en el dataset: ", cantNotSarcastica/cantTotal)

Proporción comentarios sarcasticos en AskReddit:  0.7244148337708751
Proporción comentarios no sarcasticos en AskReddit:  0.2755851662291248
Proporción comentarios sarcasticos en funny:  0.41814876110113985
Proporción comentarios no sarcasticos en funn:  0.5818512388988601
Proporción comentarios sarcasticos en politics:  0.6133215306249223
Proporción comentarios no sarcasticos en politics:  0.38667846937507766
Proporción comentarios sarcasticos en leagueoflegends:  0.4574697144348507
Proporción comentarios no sarcasticos en leagueoflegends:  0.5425302855651493
Proporción comentarios sarcasticos en worldnews:  0.5142723444080487
Proporción comentarios no sarcasticos en worldnews:  0.48572765559195136
Proporción comentarios sarcasticos en el dataset:  0.5775935071709719
Proporción comentarios no sarcasticos en el dataset:  0.42240649282902804


# Vectorización

**Tokenización**: convertir un párrafo u oración a unidades (tokens), usualmente cada palabra es un token. 

En este caso, nuestra función `tokenize` es bastante simple (e ineficiente), pero sirve para nuestros simple propósito.

**Stopword removal**: eliminar tokens irrelevantes, palabras comunes y a veces signos de puntuación.

En nuestro caso, únicamente estamos eliminando los símbolos de puntuación con ayuda del set `punctuation`.

In [8]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'ea

In [9]:
import string
punctuation = set(string.punctuation)
digits = set(string.digits)
words=stopwords.words('english')

def tokenize(sentence):
    tokens = []
    for token in sentence.split():
        new_token = []
        #if (token not in words):
        for character in token:
            if (character not in punctuation) and (character not in digits) :
                new_token.append(character.lower())
        if new_token:
            tokens.append("".join(new_token))
    return tokens

In [10]:
tokenize("Go until jurong point, crazy, i love you 1313")

['go', 'until', 'jurong', 'point', 'crazy', 'i', 'love', 'you']

In [11]:
yes_or_no.head()["comment"].apply(tokenize)

0    [i, dont, pay, attention, to, her, but, as, lo...
1    [trick, or, treating, in, general, is, just, w...
2                                    [what, the, fuck]
3                         [this, would, make, me, cry]
4    [my, stuffed, animal, ive, had, since, i, was,...
Name: comment, dtype: object

**Stemming/Lemmatization**: Convertir cada token a su forma base: {“biblioteca”, “bibliotecario”, ”bibliotecas”} → “bibliotec”.

En nuestro caso no estamos haciendo este paso, pero si es necesario, puedes revisar cosas como [NLTK - stemming](https://pythonspot.com/nltk-stemming/) o [Lemmatization Approaches with Examples in Python](https://www.machinelearningplus.com/nlp/lemmatization-examples-python/)

**One-Hot encoding**: después de la tokenización, poner en una tabla todos los tokens en el vocabulario y por cada ocurrencia de un token en un texto, marcar con un 1 en la fila correspondiente, por ejemplo considerando las dos frases siguientes:

 1. Call FREEPHONE 0800 542 0578 now!
 2. Did you call me just now ah?
 
Obtendríamos algo como esto:
 
|       | 0578 | 0800 | 542 | ah | call | did | freephone | just | me | now | you |
|-------|------|------|-----|----|------|-----|-----------|------|----|-----|-----|
| **1** | 1    | 1    | 1   | 0  | 1    | 0   | 1         | 0    | 0  | 1   | 0   |
| **2** | 0    | 0    | 0   | 1  | 1    | 1   | 0         | 1    | 1  | 1   | 1   |  

Aquí es donde entra **Scikit-Learn** a través de la clase `CountVectorizer` del módulo `sklearn.feature_extraction.text`.

[Slides]

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

# Pequeño Ejemplo

In [13]:
demo_vectorizer = CountVectorizer(
    tokenizer = tokenize,
    binary=True
)

Explicación de los parámetros:  

 - **tokenizer = tokenize**: `CountVectorizer` tiene un tokenizador por default, al pasarle nuestra función lo estamos reemplazando con el que nosotros escribimos.  
 - **binary = True**: `CountVectorizer` por default en lugar de `1` cuenta el número de ocurrencias de cada token, al establecer `binary = True`, le estamos indicando que no importa cuantas veces ocurra una palabra, solamente la debe contar una vez

In [14]:
examples = [
    "Call FREEPHONE 0800 542 0578 now!",
    "Did you call me just now ah?"
]
demo_vectorizer.fit(examples)
vectors = demo_vectorizer.transform(examples).toarray()

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


Usamos `fit` y `transform` de manera separada, aunque en este caso pudimos haber usado `fit_transform`.

**Nota**: usamos `toarray` para obtener un un *numpy array* ya que por default `transform` devuelve una [matriz dispersa](https://en.wikipedia.org/wiki/Sparse_matrix) que, mientras que es buena para no consumir memoria, no es tan amigable para mostrar cómo es que se ven los datos.

In [15]:
headers = sorted(demo_vectorizer.vocabulary_.keys())
pd.DataFrame(vectors, columns=headers)

,ah,call,did,freephone,just,me,now,you
0,0,1,0,1,0,0,1,0
1,1,1,1,0,1,1,1,1


#Resultado preliminar:

In [16]:
from sklearn.model_selection import train_test_split

## Clasificadores para AskReddit


In [17]:

datosAskReddit=yes_or_no[yes_or_no['subreddit']=="AskReddit"]
datosAskReddit.head()


,comment,subreddit,score,sarcasm,Llave,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,"I don't pay attention to her, but as long as s...",AskReddit,0,yes,AskReddityes,,AskReddityes,65674.0,0.724415,NaN,sarcasm,170516.0,0.577594
1,Trick or treating in general is just weird...,AskReddit,1,yes,AskReddityes,NaN,AskRedditno,24984.0,0.275585,NaN,nosarcasm,124702.0,0.422406
2,what the fuck,AskReddit,22,yes,AskReddityes,NaN,NaN,90658.0,NaN,NaN,NaN,295218.0,NaN
3,This would make me cry.,AskReddit,1,yes,AskReddityes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,My stuffed animal I've had since I was born.,AskReddit,1,yes,AskReddityes,NaN,funnyyes,17939.0,0.418149,NaN,NaN,NaN,NaN


In [18]:
train_comment_A,test_comment_A, train_sarcasm_A, test_sarcasm_A = train_test_split(datosAskReddit["comment"], 
                                                                    datosAskReddit["sarcasm"],
                                                                    test_size=.33, random_state=37,
                                                                    stratify=datosAskReddit["sarcasm"])

print(f"Training examples: {len(train_comment_A)}, testing examples {len(test_comment_A)}")


Training examples: 60740, testing examples 29918


In [19]:
real_vectorizer = CountVectorizer(tokenizer = tokenize, binary=True)
train_X_A = real_vectorizer.fit_transform(train_comment_A)
test_X_A = real_vectorizer.transform(test_comment_A)
train_X_A.shape

(60740, 39822)

###Usando Arbol de decision

In [20]:
classifier_Tree_Ask = DecisionTreeClassifier()
classifier_Tree_Ask.fit(train_X_A, train_sarcasm_A)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [21]:
from sklearn.metrics import accuracy_score
predicciones_Tree_Ask = classifier_Tree_Ask.predict(test_X_A)
accuracy_Tree_Ask = accuracy_score(test_sarcasm_A, predicciones_Tree_Ask)
print(f"Accuracy: {accuracy_Tree_Ask:.4%}")

Accuracy: 76.1314%


###Usando linearSvc

In [22]:
classifier_Svc_Ask = LinearSVC( max_iter=300000)
classifier_Svc_Ask.fit(train_X_A, train_sarcasm_A)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=300000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [23]:
from sklearn.metrics import accuracy_score
predicciones_Svc_Ask = classifier_Svc_Ask.predict(test_X_A)
accuracy_Svc_Ask = accuracy_score(test_sarcasm_A, predicciones_Svc_Ask)
print(f"Accuracy: {accuracy_Svc_Ask:.4%}")

Accuracy: 80.3764%


## Clasificadores para politics


In [24]:

datosPolitics=yes_or_no[yes_or_no['subreddit']=="politics"]
datosPolitics.head()


,comment,subreddit,score,sarcasm,Llave,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
104647,NC and NH.,politics,2,yes,politicsyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104648,I think a significant amount would be against ...,politics,92,yes,politicsyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104649,because it's what really bothers him... and it...,politics,15,yes,politicsyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104650,Conservatism as an ideology is for sure a reac...,politics,1,yes,politicsyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104651,"Maybe not control, but certainly that is evide...",politics,1,yes,politicsyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
train_comment_P,test_comment_P, train_sarcasm_P, test_sarcasm_P = train_test_split(datosPolitics["comment"], 
                                                                    datosPolitics["sarcasm"],
                                                                    test_size=.33, random_state=37,
                                                                    stratify=datosPolitics["sarcasm"])

print(f"Training examples: {len(train_comment_P)}, testing examples {len(test_comment_P)}")


Training examples: 43142, testing examples 21250


In [26]:
real_vectorizer = CountVectorizer(tokenizer = tokenize, binary=True)

train_X_P = real_vectorizer.fit_transform(train_comment_P)
test_X_P = real_vectorizer.transform(test_comment_P)

train_X_P.shape

(43142, 30975)

###Usando Arbol de decision

In [27]:
classifier_Tree_P = DecisionTreeClassifier()
classifier_Tree_P.fit(train_X_P, train_sarcasm_P)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [28]:
from sklearn.metrics import accuracy_score
predicciones_Tree_P = classifier_Tree_P.predict(test_X_P)
accuracy_Tree_P = accuracy_score(test_sarcasm_P, predicciones_Tree_P)
print(f"Accuracy: {accuracy_Tree_P:.4%}")

Accuracy: 75.0635%


###Usando linearSvc

In [29]:
classifier_Svc_P = LinearSVC( max_iter=300000)
classifier_Svc_P.fit(train_X_P, train_sarcasm_P)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=300000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [30]:
from sklearn.metrics import accuracy_score
predicciones_Svc_P = classifier_Svc_P.predict(test_X_P)
accuracy_Svc_P = accuracy_score(test_sarcasm_P, predicciones_Svc_P)
print(f"Accuracy: {accuracy_Svc_P:.4%}")

Accuracy: 81.8871%


# Pregunta 3

### Usando LinearSvc con askreddit en entramiento y polits pa testing


In [31]:
print(test_X_P.shape)
print(test_X_A.shape)

predicciones_PA = classifier_Svc_P.predict(test_X_A[:,0:30975])
accuracy_PA = accuracy_score(test_sarcasm_A, predicciones_PA)
print(f"Accuracy: {accuracy_PA:.4%}")

(21250, 30975)
(29918, 39822)
Accuracy: 69.9512%


### Predicciones en nuevos datos

In [32]:
examples=["the penguin loves nicolas",
          "I love my life and my dog",
          "i like the idea but not the execution",
          "the sky is blue",
          "great idea",
          "chile has many earthquakes",
          "i was looking for this comment",
          "my mother died yesterday",
          "now I have anal cancer",
          "trump is the worst president of USA",
          "Republicans will take credit once it's done.",
          "Fuck this",
          "the huracan maria destroyed puerto rico",
          "Hilarious that Russia’s statements intended to back up Belarus basically say that it is exactly what it looks like, but no big deal."
          ]
#for i in test_text:
 # examples.append(i)

examples_X = real_vectorizer.transform(examples)
predicciones = classifier.predict(examples_X)

NameError: ignored

In [ ]:
for text, label in zip(examples, predicciones):
    print(f"{label:5} - {text}")